---
title: Combine magnetic field data and plasma data
---

- combine features from different sources/instruments (magnetic field, state data, etc.)
- generate new features


In [1]:
# | default_exp integration
# | export
import polars as pl
import polars.selectors as cs
from space_analysis.plasma.formulary import df_Alfven_speed as compute_Alfven_speed

In [2]:
# | export


def interpolate(df1: pl.DataFrame, df2, on="time"):

    df = (
        pl.concat([df1, df2], how="diagonal")
        .to_pandas()
        .set_index(on)
        .sort_index()
        .interpolate(method="index", limit_area='inside')
        .reset_index()
    )

    return pl.from_pandas(df)


def combine_features(
    events: pl.DataFrame,
    states_data: pl.DataFrame,
    subset_cols: list = [
        "plasma_density",
        "plasma_temperature",
        "plasma_speed",
    ],
    method: str = "interpolate",
):
    # change time format: see issue: https://github.com/pola-rs/polars/issues/12023

    states_data = states_data.with_columns(
        cs.datetime().dt.cast_time_unit("ns"),
    ).sort("time")

    events = events.with_columns(
        cs.datetime().dt.cast_time_unit("ns"),
    ).sort("time")

    subset_cols = subset_cols + [
        "time"
    ]  # https://stackoverflow.com/questions/2347265/why-does-behave-unexpectedly-on-lists
    states_data_subset = states_data.select(subset_cols)
    df = events.join_asof(states_data, on="time", strategy="nearest")

    if method == "interpolate":
        before_df = interpolate(df.select(time=pl.col("d_tstart")), states_data_subset)
        after_df = interpolate(df.select(time=pl.col("d_tstop")), states_data_subset)
        return (
            df.sort("d_tstart")
            .join(
                before_df,
                left_on="d_tstart",
                right_on="time",
                suffix="_before",
            )
            .sort("d_tstop")
            .join(
                after_df,
                left_on="d_tstop",
                right_on="time",
                suffix="_after",
            )
        )

    elif method == "nearest":
        return (
            df.sort("d_tstart")
            .join_asof(
                states_data_subset,
                left_on="d_tstart",
                right_on="time",
                strategy="backward",
                suffix="_before",
            )
            .sort("d_tstop")
            .join_asof(
                states_data_subset,
                left_on="d_tstop",
                right_on="time",
                strategy="forward",
                suffix="_after",
            )
        )
    else:
        return df

## Additional features after combining

With combined dataset, we calculate additional features for each candidate.

Length

the length along the n direction of LMN coordinate system.

$$L_{n} = v_{n}  T_{duration}$$

However this may not be accurate due to the MVA method.

$$L_{mn} = v_{mn}  T_{duration}$$

If we have the normal vector of the current sheet, we can calculate the length along the normal direction.

$$L_{normal} = L_{k} = v_{normal}  T_{duration}$$

Additionally, we can calculate the length projected into RTN coordinate system.

$$L_{R} = L_{k} \cos \theta$$

$$ j*0 = (\frac{d B}{d t})*{max} \frac{1}{v\_{mn}}$$


In [3]:
# | export
import astropy.units as u
from astropy.constants import mu0, e
from plasmapy.formulary.lengths import inertial_length
from xarray_einstats import linalg
from discontinuitypy.utils.basic import df2ts
from discontinuitypy.core.propeties import calc_rotation_angle
import xarray as xr

  File "/Users/zijin/micromamba/envs/discontinuitypy/lib/python3.10/site-packages/pdpipe/__init__.py", line 105, in <module>
    from . import nltk_stages
  File "/Users/zijin/micromamba/envs/discontinuitypy/lib/python3.10/site-packages/pdpipe/nltk_stages.py", line 19, in <module>
    import nltk
ModuleNotFoundError: No module named 'nltk'



In [4]:
# | export
def vector_project(v1: xr.DataArray, v2: xr.DataArray, dim="v_dim"):
    return xr.dot(v1, v2, dims=dim) / linalg.norm(v2, dims=dim)


def vector_project_pl(df: pl.DataFrame, v1_cols, v2_cols, name=None):

    v1 = df2ts(df, v1_cols).assign_coords(v_dim=["x", "y", "z"])
    v2 = df2ts(df, v2_cols).assign_coords(v_dim=["x", "y", "z"])
    result = vector_project(v1, v2, dim="v_dim")

    return df.with_columns(pl.Series(result.data).alias(name or "v_proj"))

In [17]:
# test
import numpy as np

da_a = xr.DataArray(np.arange(3 * 2).reshape(3, 2), dims=["v_dim", "time"])
print(da_a)
vector_project(da_a, da_a, dim="v_dim")

<xarray.DataArray (v_dim: 3, time: 2)>
array([[0, 1],
       [2, 3],
       [4, 5]])
Dimensions without coordinates: v_dim, time


<xarray.DataArray (time: 2)>
array([4.47213595, 5.91607978])
Dimensions without coordinates: time

In [ ]:
# | export
def calc_rotation_angle_pl(df: pl.DataFrame, v1_cols, v2_cols, name):
    v1 = df.select(v1_cols).to_numpy()
    v2 = df.select(v2_cols).to_numpy()

    result = calc_rotation_angle(v1, v2)

    return df.with_columns(pl.Series(result).alias(name))

### Inertial length


In [ ]:
# | export
def compute_inertial_length(df: pl.DataFrame, density_col="plasma_density"):

    density = df[density_col].to_numpy() * u.cm ** (-3)
    result = inertial_length(density, "H+").to(u.km)

    return df.with_columns(ion_inertial_length=pl.Series(result.value))

### Alfven current


In [ ]:
# | export
def compute_Alfven_current(
    df: pl.DataFrame,
    density_col="plasma_density",
):

    Alfven_speed = df["Alfven_speed"].to_numpy() * u.km / u.s
    density = df[density_col].to_numpy() * u.cm ** (-3)

    result = (e.si * Alfven_speed * density).to(u.nA / u.m**2)

    return df.with_columns(j_Alfven=pl.Series(result.value))

## All features


In [ ]:
# | export
def calc_plasma_parameter_change(
    df: pl.DataFrame,
    temperature_col: str = None,
):
    if temperature_col:
        df = df.rename(
            {
                f"{temperature_col}_before": "T.before",
                f"{temperature_col}_after": "T.after",
            }
        ).with_columns((pl.col("T.after") - pl.col("T.before")).alias("T.change"))

    return (
        df.rename(
            {
                "plasma_density_before": "n.before",
                "plasma_speed_before": "v.ion.before",
                "plasma_density_after": "n.after",
                "plasma_speed_after": "v.ion.after",
            }
        )
        .pipe(
            compute_Alfven_speed,
            B="B.before",
            n="n.before",
            col_name="v.Alfven.before",
            sign=True,
        )
        .pipe(
            compute_Alfven_speed,
            B="B.after",
            n="n.after",
            col_name="v.Alfven.after",
            sign=True,
        )
        .pipe(
            compute_Alfven_speed,
            n="n.before",
            B="B.vec.before.l",
            col_name="v.Alfven.before.l",
            sign=True,
        )
        .pipe(
            compute_Alfven_speed,
            n="n.after",
            B="B.vec.after.l",
            col_name="v.Alfven.after.l",
            sign=True,
        )
        .with_columns(
            (pl.col("n.after") - pl.col("n.before")).alias("n.change"),
            (pl.col("v.ion.after") - pl.col("v.ion.before")).alias("v.ion.change"),
            (pl.col("v.ion.after.l") - pl.col("v.ion.before.l")).alias(
                "v.ion.change.l"
            ),
            (pl.col("B.after") - pl.col("B.before")).alias("B.change"),
            (pl.col("v.Alfven.after") - pl.col("v.Alfven.before")).alias(
                "v.Alfven.change"
            ),
            (pl.col("v.Alfven.after.l") - pl.col("v.Alfven.before.l")).alias(
                "v.Alfven.change.l"
            ),
        )
    )

In [ ]:
# | export
from discontinuitypy.utils.polars import decompose_vector


def calc_combined_features(
    df: pl.DataFrame,
    vec_cols: list[str] = ["v_x", "v_y", "v_z"],
    normal_cols: list[str] = ["k_x", "k_y", "k_z"],
    b_cols: list[str] = None,  # ["B_background_x", "B_background_y", "B_background_z"]
    density_col="plasma_density",
    detail: bool = True,
    Vl_cols=["Vl_x", "Vl_y", "Vl_z"],
    Vn_cols=["Vn_x", "Vn_y", "Vn_z"],
    thickness_cols=["L_k"],
    current_cols=["j0_k"],
    temperature_col: str = None,
    duration_method="distance",
):
    """_summary_

    Args:
        df (pl.DataFrame): _description_
        vec_cols (list[str], optional): # plasma velocity vector in any fixed coordinate system. Defaults to [ "v_x", "v_y", "v_z", ].
        normal_cols (list[str], optional): normal vector of the discontinuity plane. Defaults to [ "k_x", "k_y", "k_z", ].
        detail (bool, optional): _description_. Defaults to True.
        Vl_cols (list, optional): maxium variance direction vector of the magnetic field. Defaults to [ "Vl_x", "Vl_y", "Vl_z", ].
        Vn_cols (list, optional): minimum variance direction vector of the magnetic field. Defaults to [ "Vn_x", "Vn_y", "Vn_z", ].
        current_cols (list, optional): _description_. Defaults to ["j0_mn", "j0_k"].
    """

    length_norm = pl.col("ion_inertial_length")
    current_norm = pl.col("j_Alfven")
    b_norm = pl.col("b_mag")

    j_factor = ((u.nT / u.s) * (1 / mu0 / (u.km / u.s))).to(u.nA / u.m**2)

    result = df

    if b_cols is not None:
        result = result.pipe(
            calc_rotation_angle_pl, b_cols, normal_cols, name="theta_n_b"
        )

    result = (
        result.pipe(vector_project_pl, vec_cols, Vl_cols, name="v_l")
        .pipe(vector_project_pl, vec_cols, Vn_cols, name="v_n")
        .pipe(vector_project_pl, vec_cols, normal_cols, name="v_k")
        .with_columns(
            pl.col("v_n").abs(),
            pl.col("v_k").abs(),
            v_mn=(pl.col("plasma_speed") ** 2 - pl.col("v_l") ** 2).sqrt(),
        )
        .with_columns(
            L_k=pl.col("v_k") * pl.col("duration"),
            j0_k=pl.col("d_star") / pl.col("v_k"),
            # NOTE: n direction is not properly determined for MVA analysis
            # j0_mn=pl.col("d_star") / pl.col("v_mn"),
            # L_n=pl.col("v_n") * pl.col("duration"),
            # L_mn=pl.col("v_mn") * pl.col("duration"),
            # NOTE: the duration is not properly determined for `max distance` method
            # L_k=pl.col("v_k") * pl.col("duration"),
        )
        .pipe(compute_inertial_length)
        .pipe(compute_Alfven_speed, n=density_col, B="b_mag")
        .pipe(compute_Alfven_current)
        .with_columns(
            cs.by_name(current_cols) * j_factor.value,
        )
        .with_columns(
            (cs.by_name(thickness_cols) / length_norm).name.suffix("_norm"),
            (cs.by_name(current_cols) / current_norm).name.suffix("_norm"),
            (cs.by_name(b_cols) / b_norm).name.suffix("_norm"),
        )
    )

    if detail:
        result = (
            result.pipe(
                vector_project_pl,
                [_ + "_before" for _ in vec_cols],
                Vl_cols,
                name="v.ion.before.l",
            )
            .pipe(
                vector_project_pl,
                [_ + "_after" for _ in vec_cols],
                Vl_cols,
                name="v.ion.after.l",
            )
            .pipe(decompose_vector, "B.vec.before", suffixes=[".l", ".m", ".n"])
            .pipe(decompose_vector, "B.vec.after", suffixes=[".l", ".m", ".n"])
            .pipe(calc_plasma_parameter_change, temperature_col=temperature_col)
        )

    return result

In [4]:
# | hide
from nbdev import nbdev_export

nbdev_export()